## Step 1 Clone repository given by seedVC

# SEED-VC Voice Conversion SERVER

If Google ask you to restart session, just ignore it

In [1]:
!git clone https://github.com/Plachtaa/seed-vc
%cd seed-vc
!pip install -r requirements.txt
!pip uninstall tensorflow -y

Cloning into 'seed-vc'...
remote: Enumerating objects: 907, done.
remote: Counting objects: 100% (374/374), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 907 (delta 297), reused 253 (delta 253), pack-reused 533 (from 1)
Receiving objects: 100% (907/907), 66.27 MiB | 19.42 MiB/s, done.
Resolving deltas: 100% (470/470), done.
/content/seed-vc
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.

Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0


# Task
Create a Flask server that listens for POST requests containing base64 encoded audio data in the request body. Decode the base64 audio data, save it to a temporary file, and use this file as the `--source` argument when running the inference script.

In [2]:
pip install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 50.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboardx 2.6 requires protobuf<4,>=3.8.0, but you have protobuf 6.32.0 which is incompatible.
descript-audiotools 0.7.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 6.32.0 which is incompatible.
tensorflow-text 2.19.0 requires tensorflo

## Install flask and required libraries

### Subtask:
Install Flask and any other libraries needed to handle the base64 decoding and file operations.


**Reasoning**:
Install Flask to set up the web server for handling POST requests. base64 and tempfile are part of the standard library and do not need explicit installation.



In [3]:
!pip install Flask pyngrok

## Create flask application

### Subtask:
Set up a basic Flask application with a route to handle the incoming POST request.


**Reasoning**:
Set up a basic Flask application with a route to handle the incoming POST request as instructed.



In [ ]:
from google.colab import files
import os

uploaded = files.upload()

uploaded_file_path = None
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  # Assuming the file is saved in the current directory
  uploaded_file_path = os.path.join(os.getcwd(), fn)
  print(f"Uploaded file path: {uploaded_file_path}")

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import base64
import tempfile
import subprocess
import os

# Assuming uploaded_file_path is defined in a previous cell
if 'uploaded_file_path' not in globals() or uploaded_file_path is None:
    # Handle the case where the file hasn't been uploaded yet
    # You might want to return an error or use a default path
    print("Error: Reference audio file not uploaded yet. Please run the upload cell first.")
    # For now, I will use a placeholder and the request will likely fail
    target_audio_path = "/content/default_target.wav"
else:
    target_audio_path = uploaded_file_path

!ngrok config add-authtoken 2wSXD9affKGSze1RTtVyPfEaz3A_2Q7oVURk8StjmkQaswTKK

application = Flask(__name__)
@application.route('/', methods=['POST'])
def handle_post_request():
    temp_audio_path = None
    try:
        data = request.get_json()
        if 'audio_data' not in data:
            return jsonify({"error": "Missing 'audio_data' in request body"}), 400

        audio_data_base64 = data['audio_data']
        audio_bytes = base64.b64decode(audio_data_base64)

        # Save decoded audio to a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp_audio_file:
            temp_audio_file.write(audio_bytes)
            temp_audio_path = temp_audio_file.name

        print(f"Decoded audio saved to temporary file: {temp_audio_path}")

        # Define inference command
        # Assuming the inference script is at /content/seed-vc/inference.py
        # and the output directory is ./reconstructed relative to the script
        inference_command = [
            "python", "inference.py",
            "--source", temp_audio_path,
            "--target", target_audio_path, # Use the uploaded file path as target
            "--output", "./reconstructed",
            "--diffusion-steps", "25",
            "--length-adjust", "1.0",
            "--inference-cfg-rate", "0.7"
            # Add config and checkpoint paths if needed for fine-tuned model
            # "--config-path", "./runs/Kobe_Taffy_ft/config_dit_mel_seed_uvit_whisper_small_wavenet.yml",
            # "--checkpoint-path", "./runs/Kobe_Taffy_ft/ft_model.pth"
        ]

        # Execute inference script
        # Change cwd to the seed-vc directory
        process = subprocess.run(inference_command, capture_output=True, text=True, cwd="/content/seed-vc")
        print("Inference script stdout:", process.stdout)
        print("Inference script stderr:", process.stderr)

        if process.returncode != 0:
            return jsonify({"error": "Inference script failed", "details": process.stderr}), 500

        # Assuming the inference script outputs a file named based on the inputs
        # You might need to adjust this based on the actual output filename
        # Construct the expected output filename based on the source and target filenames
        source_base = os.path.basename(temp_audio_path).replace('.wav', '')
        target_base = os.path.basename(target_audio_path).replace('.wav', '')
        output_filename = f"vc_{source_base}_{target_base}_1.0_25_0.7.wav"
        output_path = os.path.join("/content/seed-vc/reconstructed", output_filename)


        if not os.path.exists(output_path):
             # Fallback or error handling if the output filename is not as expected
             # List files in the output directory to help debugging
            output_files = os.listdir("/content/seed-vc/reconstructed")
            print(f"Expected output not found: {output_path}. Files in output dir: {output_files}")
            return jsonify({"error": f"Inference output file not found at {output_path}", "files_in_output_dir": output_files}), 500


        # Read the output audio file and encode it back to base64
        with open(output_path, "rb") as output_audio_file:
            output_audio_bytes = output_audio_file.read()
            output_audio_base64 = base64.b64encode(output_audio_bytes).decode('utf-8')

        # Clean up the inference output file
        os.remove(output_path)

        return jsonify({"converted_audio_base64": output_audio_base64}), 200

    except Exception as e:
        return jsonify({"error": f"Error processing request: {e}"}), 500
    finally:
        # Clean up the temporary input audio file
        if temp_audio_path and os.path.exists(temp_audio_path):
            os.remove(temp_audio_path)
            print(f"Temporary input file removed: {temp_audio_path}")


print(ngrok.connect(5000))

if __name__ == '__main__':
  application.run(threaded=True, port=5000)

Error: Reference audio file not uploaded yet. Please run the upload cell first.
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
NgrokTunnel: "https://6f2ceda564f8.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
